# Data Preparation - Wikipedia Indonesia Text Data

Notebook ini mendemonstrasikan proses data preparation pada data tekstual.

## Alur Proses:
1. **Load**: Membaca dan menjelajahi dataset awal
2. **Data Cleaning**: Membersihkan markup Wikipedia dan ekstrak kategori
3. **Pipeline Processing**: Menerapkan transformasi data secara bertahap
4. **Tokenization** : Memecah teks menjadi kunpulan token
5. **Vectorization**: Mengkonversi text menjadi TF-IDF matrix
6. **Search & Retrieval**: Mencari dokumen paling relevan dengan query

In [1]:
import pandas as pd
import re

## 2. Load Dataset

Membaca file CSV yang berisi data artikel Wikipedia Indonesia dalam format raw (mentah dengan markup Wiki masih ada).

In [2]:
#Membaca dataframe (file csv), dengan delimiter pipe (|)
df = pd.read_csv('wikipedia_raw.csv', delimiter = '|')

In [3]:
#menampilkan dataframe
df.head(5)

,Unnamed: 0,id,timestamp,title,text
0,0,1,2022-04-24T00:35:51Z,Asam deoksiribonukleat,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...
1,1,3,2022-05-05T11:26:32Z,Anwar Sadat,'''Muhammad Anwar el-Sadat'''; ) adalah seoran...
2,2,5,2022-09-26T04:25:33Z,Arkeologi,[[Berkas:Bulgandry Aboriginal Site.JPG|jmpl|Si...
3,3,6,2022-11-02T02:33:49Z,Antropologi,'''Antropologi''' adalah ilmu tentang manusia....
4,4,8,2022-11-16T04:21:39Z,Bahasa Indonesia,'''Bahasa Indonesia''' adalah [[bahasa nasiona...


In [4]:
#meilhat informasi umum dari dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86436 entries, 0 to 86435
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  86436 non-null  int64 
 1   id          86436 non-null  int64 
 2   timestamp   86436 non-null  object
 3   title       86436 non-null  object
 4   text        86436 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.3+ MB


#### Melihat contoh isi `teks`

In [5]:
#Menampilkan contoh isi data teks
print(df.loc[0,'text'])

[[Berkas:DNA Structure+Key+Labelled.pn NoBB.png|jmpl|ka|340px|Struktur [[heliks ganda]] DNA. [[Atom]]-atom pada struktur tersebut diwarnai sesuai dengan [[unsur kimia]]nya dan struktur detail dua pasangan basa ditunjukkan oleh gambar kanan bawah]]
[[Berkas:DNA animation.gif|jmpl|Gambaran tiga dimensi DNA]]
'''Asam deoksiribonukleat''', lebih dikenal dengan singkatan '''DNA''' ([[bahasa Inggris]]: '''''d'''eoxyribo'''n'''ucleic '''a'''cid''), adalah salah satu jenis [[asam nukleat]] yang memiliki kemampuan [[Hereditas|pewarisan]] sifat. Keberadaan asam deoksiribonukleat ditemukan di dalam [[nukleoprotein]] yang membentuk [[inti sel]]. [[James Dewey Watson]] dan [[Francis Crick]] merupakan ilmuwan pertama yang mengajukan model struktur DNA pada tahun 1953 dengan bentuk [[pilinan ganda]]. Setiap DNA tersusun dari dua buah rantai [[polinukleotida]].<ref></ref> DNA merupakan sejenis biomolekul yang menyimpan dan menyandi instruksi-instruksi [[genetika]] setiap [[organisme]] dan banyak jenis

## 4.Data Cleaning and Extraction

Membersihkan text dari markup Wikipedia dan mengekstrak informasi.
**Fungsi Utama:**
- Ekstrak kategori dari format `[[Kategori:...]]`
- Pembersihan teks lengkap (hapus ref, link, template, dsb)

In [6]:
# mengambil 1 sampel text sebagai contoh
sample_text = df.loc[0, 'text']

### 4.1 Esktrak informasi kategori

`[[Kategori:Genetika molekular|Nukleat asam DNA]]`

`[[Kategori:Asam nukleat]]`

In [7]:
# mencari dan mengekstrak informasi kategori artikel dari teks
x = re.findall(r'\[\[Kategori:(.*?)\]\]', sample_text)
kategori_extracted = []
for kategori in x:
    kategori_extracted.extend(kategori.split('|'))
print(kategori_extracted)

['Genetika molekular', 'Nukleat asam DNA', 'Asam nukleat']


In [8]:
# Modularisasi ke bentuk fungsi, agar lebih mudah digunakan pada dataframe
def extract_kategori(text):
    x = re.findall(r'\[\[Kategori:(.*?)\]\]',text)
    kategori_extracted = []
    for kategori in x:
        kategori_extracted.extend(kategori.split('|'))
    return kategori_extracted

In [9]:
# menjalankan fungsi terhadap sample_text
extract_kategori(sample_text)

['Genetika molekular', 'Nukleat asam DNA', 'Asam nukleat']

### 4.2 Membersihkan/ Menghapus markdown files
`[[Berkas:DNA animation.gif|jmpl|Gambaran tiga dimensi DNA]]{{genetika}}`

In [10]:
#mencari dan menghapus semua markdown files
sample_text_clean = re.sub(r'\[\[Berkas:.*?\n','', sample_text, flags=re.IGNORECASE)
# print(re.sub(r'\[\[Berkas:.*?\n','', sample_text, flags=re.IGNORECASE))

### 4.3 Membersihkan/Menghapus reftag
`<ref>{{cite book|last = Saenger|first = Wolfram|title = Principles of Nucleic Acid Structure|url = https://archive.org/details/...
}}</ref>`

In [11]:
#mencari dan menghapus semua reftag
sample_text_clean = re.sub(r'<ref.*?<\/ref>', '', sample_text_clean, flags=re.DOTALL)
# print(sample_text_clean)

### 4.4 Membersihkan internal link

`[[Hereditas|pewarisan]]` akan  menjadi `pewarisan` (hanya mengambil labelnya saja)

`[[kromosom]]` akan menjadu `kromosom`

In [12]:
#cleansing internal link dengan label
sample_text_clean = re.sub(r'\[\[[\w\s]*?\|([\w\s]*?)\]\]',r'\1', sample_text_clean)
# print(sample_text_clean)

In [13]:
#cleansing internal link tanpa label
sample_text_clean = re.sub(r'\[\[([\w\s]*)\]\]', r'\1', sample_text_clean)
# print(sample_text_clean)

In [14]:
def cleanse_text(text):
    text = re.sub(r'\[\[(Berkas|File).*?\n',' ',text, flags=re.IGNORECASE)
    text = re.sub(r'<ref.*?<\/ref>', ' ', text, flags=re.DOTALL)
    text = re.sub(r'\[\[[\w\s]*?\|([\w\s]*?)\]\]',r'\1',text)
    text = re.sub(r'\[\[(.*?)\]\]', r'\1', text)
    text = text.strip()
    return text

In [15]:
# mencoba fungsi pada sample_text dan membandingkan hasil sebelum dan sesudah cleansing

clean_text = cleanse_text(sample_text)

print("=== SEBELUM (Raw) ===")
print(sample_text[:300]) # Print 300 karakter pertama aja
print("\n" + "="*20 + "\n")
print("=== SESUDAH (Clean) ===")
print(clean_text[:300])

=== SEBELUM (Raw) ===
[[Berkas:DNA Structure+Key+Labelled.pn NoBB.png|jmpl|ka|340px|Struktur [[heliks ganda]] DNA. [[Atom]]-atom pada struktur tersebut diwarnai sesuai dengan [[unsur kimia]]nya dan struktur detail dua pasangan basa ditunjukkan oleh gambar kanan bawah]]
[[Berkas:DNA animation.gif|jmpl|Gambaran tiga dimens


=== SESUDAH (Clean) ===
'''Asam deoksiribonukleat''', lebih dikenal dengan singkatan '''DNA''' (bahasa Inggris: '''''d'''eoxyribo'''n'''ucleic '''a'''cid''), adalah salah satu jenis asam nukleat yang memiliki kemampuan pewarisan sifat. Keberadaan asam deoksiribonukleat ditemukan di dalam nukleoprotein yang membentuk inti s


## 5. Eksekusi Data Cleaning Pipeline

Menerapkan semua fungsi pembersihan secara berurutan menggunakan method chaining `.pipe()`.

Output: `df_cleaned` - Dataset yang sudah bersih dengan kolom text_cleaned dan kategori

**Fungsi Pipeline:**
- `start_pipeline()`: Mulai pipeline
- `extract_kategori_pipe()`:  ekstraksi kategori
- `clean_text_pipe()`:  pembersihan text
- `clean_nested_template_pipe()`: menghapus template bersarang

In [16]:
# data pipeline
def start_pipeline(df):
    return df.copy()

def extract_kategori_pipe(df):
    df['kategori'] = df['text'].apply(extract_kategori)
    return df

def cleanse_text_pipe(df):
    df['text_cleaned'] = df['text'].apply(cleanse_text)
    return df

In [17]:
#menjalankan proses pembersihan pada dataframe
df_cleaned = (df
              .pipe(start_pipeline)
              .pipe(extract_kategori_pipe)
              .pipe(cleanse_text_pipe)
              )
print(df_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86436 entries, 0 to 86435
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    86436 non-null  int64 
 1   id            86436 non-null  int64 
 2   timestamp     86436 non-null  object
 3   title         86436 non-null  object
 4   text          86436 non-null  object
 5   kategori      86436 non-null  object
 6   text_cleaned  86436 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.6+ MB
None


In [18]:
# menampilkan hasil cleansing dan ekstraksi pada dataframe


## 6. Feature Extraction

- **Tokenisasi** : Merubah teks utuh menjadi komponen-komponen token
- **Vektorisasi**: Mengkonversi text menjadi numerical feature menggunakan TF-IDF (Term Frequency-Inverse Document 

**Output**: TF-IDF matrix - menghasilkan matrix yang merupakan representasi numerik dari setiap teks/dookumen

### 6.1 Tokenisasi

#### Mengapa Tokenisasi Penting?

1. **Pemrosesan Komputer**: Algoritma machine learning memerlukan input yang terstruktur. Tokenisasi mengubah teks yang tidak terstruktur menjadi deretan token yang dapat dianalisis secara sistematis.

2. **Pengurangan Kompleksitas**: Memecah teks besar menjadi satuan kecil memudahkan pemrosesan dan analisis yang lebih efisien.

3. **Ekstraksi Fitur**: Token individual menjadi basis untuk ekstraksi fitur lanjutan seperti TF-IDF, word embeddings, dan analisis frekuensi.

#### Contoh Tokenisasi Sederhana (Word-Level Tokenization)

Teks asli:
```
"Indonesia adalah negara kepulauan terbesar di dunia yang terletak di Asia Tenggara"
```

Setelah tokenisasi dengan memisahkan berdasarkan spasi:
```
['Indonesia', 'adalah', 'negara', 'kepulauan', 'terbesar', 'di', 'dunia', 'yang', 'terletak', 'di', 'Asia', 'Tenggara']
```

Jumlah token: **12 token**

In [19]:
# simple tokenization
sample_text = 'Indonesia adalah negara kepulauan terbesar di dunia yang terletak di Asia Tenggara'

In [20]:
#mencoba tokenisasi sederhana (word level)
sample_text.split(' ')

['Indonesia',
 'adalah',
 'negara',
 'kepulauan',
 'terbesar',
 'di',
 'dunia',
 'yang',
 'terletak',
 'di',
 'Asia',
 'Tenggara']

### 6.2 Vektorisasi Teks (Text Vectorization)

**Vektorisasi** adalah proses fundamental dalam *Natural Language Processing* (NLP) untuk mengubah data teks tidak terstruktur menjadi representasi numerik (vektor) yang dapat dipahami oleh mesin.

Bayangkan vektorisasi sebagai **penerjemah**: Komputer tidak mengerti kata "Apel" atau "Jeruk", tetapi mereka mengerti bahwa `[1, 0]` berbeda dengan `[0, 1]`.

#### Mengapa Vektorisasi Penting?

Komputer bekerja dengan operasi aljabar linier. Tanpa vektorisasi, algoritma machine learning tidak dapat memproses input teks.
1.  **Kompatibilitas Matematis**: Algoritma seperti Regresi, SVM, atau Neural Networks membutuhkan input berupa array angka (matriks).
2.  **Menangkap Makna (Semantik)**: Teknik vektorisasi yang baik tidak hanya mengubah kata jadi angka, tapi juga mempertahankan informasi penting seperti seberapa unik kata tersebut dalam sebuah dokumen.
3.  **Pengukuran Jarak**: Memungkinkan kita menghitung *similarity* (kemiripan) antar dokumen menggunakan rumus geometri seperti *Cosine Similarity* atau *Euclidean Distance*.

---

#### Beberapa Metode Vektorisasi

Berikut adalah beberapa pendekatan populer, diurutkan dari yang paling sederhana hingga metode yang kami pilih:

##### 1. One-Hot Encoding
Setiap kata dalam *vocabulary* diwakili oleh satu bit unik.
* **Konsep**: Jika kita memiliki 1.000 kata unik, setiap kata adalah vektor sepanjang 1.000 dimensi dengan satu angka `1` dan sisanya `0`.
* **Kelemahan**: Menghasilkan dimensi yang sangat besar (*high dimensionality*) dan tidak menangkap frekuensi kata.

##### 2. Bag of Words (BoW) / Count Vectorizer
Metode ini menghitung **frekuensi** kemunculan kata dalam dokumen tanpa memperdulikan urutan.
* **Analogi**: Bayangkan sebuah dokumen dipotong-potong per kata, lalu dimasukkan ke dalam kantong (bag). Kita hanya menghitung jumlah kata "makan" ada berapa, "tidur" ada berapa.
* **Kelemahan**: Kata umum seperti "yang", "dan", "di" akan memiliki nilai (bobot) yang sangat besar, padahal kata-kata tersebut minim informasi unik.

##### 3. TF-IDF (Term Frequency - Inverse Document Frequency)
Metode ini adalah penyempurnaan dari BoW. TF-IDF memberikan bobot pada kata berdasarkan dua prinsip:
1.  **Seberapa sering** kata muncul di dokumen ini? (TF)
2.  **Seberapa jarang** kata muncul di seluruh koleksi dokumen lain? (IDF)

Tujuannya adalah menonjolkan kata yang **sering muncul di satu dokumen tertentu, tapi jarang muncul di dokumen lain** .

---

#### Algoritma TF-IDF

Rumus matematis untuk menghitung bobot kata $t$ dalam dokumen $d$:

$$W_{t,d} = \text{TF}(t,d) \times \text{IDF}(t)$$

Dimana:

1.  **Term Frequency (TF)**: Frekuensi kata dalam dokumen tertentu.
    $$\text{TF}(t,d) = \frac{\text{Jumlah kemunculan kata } t \text{ di dokumen } d}{\text{Total kata dalam dokumen } d}$$

2.  **Inverse Document Frequency (IDF)**: Mengukur seberapa informatif sebuah kata.
    $$\text{IDF}(t) = \log \left( \frac{\text{Total Dokumen } (N)}{\text{Jumlah Dokumen yang mengandung kata } t} \right)$$

**Simulasi Perhitungan Sederhana:**

Misalkan kita memiliki data teks:
* **Doc A**: "Kucing makan ikan"
* **Doc B**: "Kucing tidur"
* **Doc C**: "Ikan berenang"

Nilai bobot untuk kata **"Kucing"** di **Doc A**:

1.  **TF ("Kucing", Doc A)**: Muncul 1 kali dari 3 total kata = $1/3 \approx 0.33$
2.  **IDF ("Kucing")**: Muncul di 2 dokumen (A & B) dari total 3 dokumen.
    $$\text{IDF} = \log(3/2) \approx 0.176$$
3.  **TF-IDF**: $0.33 \times 0.176 = \mathbf{0.058}$

Bandingkan dengan kata **"Berenang"** di **Doc C**:
1.  **TF**: $1/2 = 0.5$
2.  **IDF**: Muncul hanya di 1 dokumen. $\log(3/1) \approx 0.477$
3.  **TF-IDF**: $0.5 \times 0.477 = \mathbf{0.238}$

> **Insight**: Kata "Berenang" memiliki bobot lebih tinggi daripada "Kucing" karena "Berenang" lebih spesifik/langka dalam koleksi dokumen ini.

---

#### Output Vektorisasi: Sparse Matrix

Hasil dari proses ini adalah sebuah matriks di mana:
* **Baris**: Dokumen (Artikel Wikipedia)
* **Kolom**: Kata (Fitur/Vocabulary)
* **Nilai**: Skor TF-IDF

Karena sebagian besar kata tidak muncul di setiap dokumen, matriks ini akan didominasi oleh angka nol. Untuk menghemat memori, ini disimpan sebagai **Sparse Matrix**.

**Ilustrasi Matriks TF-IDF:**

| Dokumen | 'indonesia' | 'adalah' | 'kepulauan' | 'teknologi' | 'ekonomi' |
| :--- | :---: | :---: | :---: | :---: | :---: |
| **Artikel 1** | 0.45 | 0.02 | 0.65 | 0.00 | 0.00 |
| **Artikel 2** | 0.00 | 0.02 | 0.00 | 0.55 | 0.35 |
| **Artikel 3** | 0.30 | 0.01 | 0.00 | 0.00 | 0.75 |

*Catatan: Nilai 0.00 merepresentasikan kata tersebut tidak ada dalam dokumen.*

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [22]:
# membuat daftar stop_words
stop_words = ['yang', 'dan', 'ini', 'itu']

In [23]:
# vektorisasi menggunakan TfIdf
vectorizer = TfidfVectorizer(lowercase=True, dtype=np.float32, max_features=50000, stop_words=stop_words)
vectorizer.fit_transform(df_cleaned['text_cleaned'])

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 31904468 stored elements and shape (86436, 50000)>

In [24]:
#mlihat daftar vocabulary
vectorizer.vocabulary_

{'asam': np.int64(4998),
 'lebih': np.int64(25816),
 'dikenal': np.int64(12795),
 'dengan': np.int64(11912),
 'singkatan': np.int64(42357),
 'dna': np.int64(13879),
 'bahasa': np.int64(5737),
 'inggris': np.int64(20626),
 'cid': np.int64(10029),
 'adalah': np.int64(2921),
 'salah': np.int64(40145),
 'satu': np.int64(40544),
 'jenis': np.int64(21662),
 'nukleat': np.int64(32688),
 'memiliki': np.int64(28692),
 'kemampuan': np.int64(23376),
 'pewarisan': np.int64(36250),
 'sifat': np.int64(42126),
 'keberadaan': np.int64(22977),
 'ditemukan': np.int64(13663),
 'di': np.int64(12197),
 'dalam': np.int64(11358),
 'membentuk': np.int64(28563),
 'inti': np.int64(20862),
 'sel': np.int64(41105),
 'james': np.int64(21415),
 'dewey': np.int64(12156),
 'watson': np.int64(48540),
 'francis': np.int64(16691),
 'crick': np.int64(10980),
 'merupakan': np.int64(30208),
 'ilmuwan': np.int64(20310),
 'pertama': np.int64(36057),
 'mengajukan': np.int64(29198),
 'model': np.int64(30812),
 'struktur': np.i

In [25]:
#melakukan fitting vectorizer pada data
tfidf_matrix = vectorizer.transform(df_cleaned['text_cleaned'])

In [26]:
# melhat representasi nilai sebuah token pada dokumen
token_idx = vectorizer.vocabulary_['dna'].item()
tfidf_matrix[0,token_idx]

np.float32(0.84020346)

### 6.3 Simulasi Search and Retreival Sederhana

Mencoba menemukan dokumen yang paling relevan berdasarkan query (kata kunci)

In [27]:
# query
query = 'prabowo gerindra subianto'

In [28]:
#melakukan tranformasi pada query (tokenisasi dan vektorisasi)
query_vec = vectorizer.transform([query])
query_vec

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 3 stored elements and shape (1, 50000)>

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

#menghitung kemiripan query ke kumpulan dokumen
similarity = cosine_similarity(query_vec, tfidf_matrix)

# top 10 dokumen yang paling mirip
top_10_idx = similarity.flatten().argsort()[::-1][:10]

#menampilkan 10 dokumen yang paling mirip
result = df_cleaned.loc[top_10_idx, :]

In [30]:
result

,Unnamed: 0,id,timestamp,title,text,kategori,text_cleaned
19045,19045,281001,2022-11-08T16:43:54Z,Prabowo Subianto,[[Letnan Jenderal]] [[TNI]] ([[Purnawirawan|Pu...,"[Pengusaha Indonesia, Pengusaha Jawa, Tokoh mi...",Letnan Jenderal TNI (Purn.) Haji (gelar)|H. ''...
45658,45658,1632906,2022-11-05T05:36:28Z,Edhy Prabowo,"[[Doktor|Dr.]] '''Edhy Prabowo''', [[Sarjana E...","[Tokoh dari Muara Enim, Politikus Indonesia, P...","Doktor|Dr. '''Edhy Prabowo''', Sarjana Ekonomi..."
19844,19844,297680,2022-11-05T09:38:04Z,Partai Gerakan Indonesia Raya,'''Partai Gerakan Indonesia Raya''' ('''Gerind...,"[Partai Gerakan Indonesia Raya, , Partai poli...",'''Partai Gerakan Indonesia Raya''' ('''Gerind...
49223,49223,1804693,2022-05-13T23:57:12Z,Daftar aksi Koalisi Merah Putih,Berikut ini adalah daftar aksi [[Koalisi Merah...,[Tokoh Koalisi Merah Putih],Berikut ini adalah daftar aksi Koalisi Merah P...
67816,67816,2791246,2022-11-10T22:08:12Z,Hasil survei pemilihan umum Presiden Indonesia...,Berikut adalah '''hasil survei pemilihan umum ...,[Pemilihan umum Presiden Indonesia 2014],Berikut adalah '''hasil survei pemilihan umum ...
64703,64703,2666062,2022-11-13T08:04:59Z,Koalisi Indonesia Adil Makmur,'''Koalisi Indonesia Adil Makmur''' adalah [[k...,"[Koalisi partai politik di Indonesia, Adil Mak...",'''Koalisi Indonesia Adil Makmur''' adalah koa...
43927,43927,1497283,2022-11-12T22:04:39Z,Moekhlas Sidik,[[Laksamana Madya]] [[TNI]] ([[Purnawirawan|Pu...,[Tokoh Tentara Nasional Indonesia Angkatan Lau...,Laksamana Madya TNI (Purnawirawan|Purn.) '''Mo...
38839,38839,1098211,2022-11-06T02:23:09Z,Andre Rosiade,"[[Haji (gelar)|H.]] '''Andre Rosiade''', [[Sar...","[Politikus Partai Gerakan Indonesia Raya, Peng...","Haji (gelar)|H. '''Andre Rosiade''', Sarjana E..."
45402,45402,1610110,2022-10-15T09:03:45Z,Koalisi Merah Putih,'''Koalisi Merah Putih''' (sering disingkat ''...,"[Koalisi partai politik di Indonesia, Merah Pu...",'''Koalisi Merah Putih''' (sering disingkat ''...
67813,67813,2791117,2019-05-03T08:52:53Z,Kasus kebohongan Ratna Sarumpaet,'''[[Ratna Sarumpaet]]''' adalah salah satu an...,"[Indonesia dalam tahun 2018, Penipuan]",'''Ratna Sarumpaet''' adalah salah satu anggot...
